<a href="https://colab.research.google.com/github/Gavesh01/CruiseShip-boarding-application/blob/main/MovieAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import required libraries
from transformers import pipeline
import pandas as pd

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# File paths (update the paths as needed)
movies_path = '/content/drive/My Drive/MovieLens/movie.csv'  # Adjust folder and file name
ratings_path = '/content/drive/My Drive/MovieLens/rating.csv'

# Load movies dataset
movies = pd.read_csv(movies_path)

# Normalize titles for consistent searching
movies['title'] = movies['title'].str.lower().str.strip()

print("Movies dataset loaded. Shape:", movies.shape)



Mounted at /content/drive
Movies dataset loaded. Shape: (27278, 3)


 Combine Movie Search and Emotion Detection

In [ ]:
def search_movies(keyword):
    keyword = keyword.lower().strip()  # Normalize keyword
    matching_movies = movies[movies['title'].str.contains(keyword, na=False)]
    return matching_movies[['movieId', 'title']]


In [ ]:
# Test the search function
keyword = "fast"
matching_movies = search_movies(keyword)
print(matching_movies)


       movieId                                              title
386        390                faster pussycat! kill! kill! (1965)
885        902                      breakfast at tiffany's (1961)
1592      1649                fast, cheap & out of control (1997)
1884      1968                         breakfast club, the (1985)
2422      2507                      breakfast of champions (1999)
3123      3210                fast times at ridgemont high (1982)
4212      4307                       fast food, fast women (2000)
4274      4369                   fast and the furious, the (2001)
5320      5417               fast runner, the (atanarjuat) (2001)
6277      6383  2 fast 2 furious (fast and the furious 2, the)...
10486    38849                            dead & breakfast (2004)
10613    40955                          breakfast on pluto (2005)
10796    43396                 world's fastest indian, the (2005)
11101    46335  fast and the furious: tokyo drift, the (fast a...
11465    4

In [ ]:
from transformers import pipeline

# Load summarization pipeline
summarizer = pipeline("summarization", model="t5-small")

# Define a function to summarize reviews
def summarize_review(review, max_length=5000, min_length=10):
    try:
        summary = summarizer(review, max_length=max_length, min_length=min_length, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        return "Error summarizing review: " + str(e)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
review = "I don't think this movie is worth much at all. Too many stupid scenes, too much bad acting, too much to appeal to a 15 year old and no one else. None of it was clever, interesting, realistic or worthwhile.The actors all have exactly one expression - anger - and they never stray far from that. I think they all took the second half of the title too seriously. Why are they furious anyway? What do they have to be furious about?Paul Walker literally sleepwalks through the entire movie and if Vin Diesel is the new action hero for the new generation, I want a recount. I liked Jordana Brewster a lot in The Faculty and was looking forward to seeing her in this, but shes wasted. Michelle Rodriguez is cementing her future as a tough girl doing movies like this - wasnt she in that female boxing movie too?The only positive thing I can say is that the movie seems to have some respect for its audience. By being rated PG-13, at least the people who would most benefit (!) from seeing this garbage would be allowed in."
print(summarize_review(review))

Your max_length is set to 5000, but your input_length is only 242. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=121)


if Vin Diesel is the new action hero for the new generation, I want a recount . the only positive thing I can say is that the movie seems to have some respect for its audience .


Use a pre-trained emotion detection model like DistilRoBERTa for analyzing user reviews.

In [ ]:
# Load sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

# Define a function to analyze sentiment
def analyze_sentiment(text):
    try:
        sentiment = sentiment_analyzer(text)
        return sentiment[0]['label'], sentiment[0]['score']
    except Exception as e:
        return "Error analyzing sentiment: " + str(e), 0


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
summary = "The storyline was gripping, the acting was top-notch."
print(analyze_sentiment(summary))

('POSITIVE', 0.9997774958610535)


Combine the search and emotion detection into a complete workflow.

In [ ]:
# Load emotion detection pipeline
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

# Define a function to detect emotions
def detect_emotion(review):
    try:
        result = emotion_classifier(review)
        dominant_emotion = max(result, key=lambda x: x['score'])
        return dominant_emotion['label'], dominant_emotion['score']
    except Exception as e:
        return "Error detecting emotion: " + str(e), 0


config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/294 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
def evaluate_emotion_accuracy(predictions, labels):
    correct = sum([1 for p, l in zip(predictions, labels) if p == l])
    accuracy = correct / len(labels)
    return accuracy

# Example Usage
predictions = ['joy', 'anger', 'sadness']
labels = ['joy', 'anger', 'sad']
accuracy = evaluate_emotion_accuracy(predictions, labels)
print(f"Emotion Detection Accuracy: {accuracy:.2f}")


Emotion Detection Accuracy: 0.67


In [ ]:
summary = "The storyline was gripping, the acting was top-notch."
print(detect_emotion(summary))

('joy', 0.9035313129425049)


In [ ]:
def movie_recommendation_workflow():
    # Step 1: Search for movies
    keyword = input("Enter a keyword to search for movies: ")
    matching_movies = search_movies(keyword)

    if matching_movies.empty:
        print("No matching movies found.")
        return

    print("\nMatching Movies:")
    for index, row in matching_movies.iterrows():
        print(f"{row['movieId']}: {row['title']}")

    # Step 2: User selects a movie
    selected_id = int(input("\nEnter the movie ID of your choice: "))
    selected_movie = matching_movies[matching_movies['movieId'] == selected_id]

    if selected_movie.empty:
        print("Invalid movie ID selected.")
        return

    print(f"You selected: {selected_movie.iloc[0]['title'].title()}")

    # Step 3: Enter and summarize the review
    review = input("Enter your review for the movie: ")
    summary = summarize_review(review)
    print("\nSummarized Review:", summary)

    # Step 4: Analyze sentiment of the summarized review
    sentiment, sentiment_score = analyze_sentiment(summary)
    print(f"Detected Sentiment: {sentiment} (Confidence: {sentiment_score:.2f})")

    # Step 5: Detect emotion from the summarized review
    emotion, emotion_score = detect_emotion(summary)
    print(f"Detected Emotion: {emotion} (Confidence: {emotion_score:.2f})")


In [ ]:
movie_recommendation_workflow()

Enter a keyword to search for movies: fast

Matching Movies:
390: faster pussycat! kill! kill! (1965)
902: breakfast at tiffany's (1961)
1649: fast, cheap & out of control (1997)
1968: breakfast club, the (1985)
2507: breakfast of champions (1999)
3210: fast times at ridgemont high (1982)
4307: fast food, fast women (2000)
4369: fast and the furious, the (2001)
5417: fast runner, the (atanarjuat) (2001)
6383: 2 fast 2 furious (fast and the furious 2, the) (2003)
38849: dead & breakfast (2004)
40955: breakfast on pluto (2005)
43396: world's fastest indian, the (2005)
46335: fast and the furious: tokyo drift, the (fast and the furious 3, the) (2006)
49347: fast food nation (2006)
54248: dog's breakfast, a (2007)
57509: sex and breakfast (2007)
59731: bigger, stronger, faster* (2008)
67923: fast & furious (fast and the furious 4, the) (2009)
69495: breakfast with scot (2007)
69945: fast and the furious, the (1955)
74673: fast lane (2009)
79197: fastest gun alive, the (1956)
81049: rooster

Your max_length is set to 5000, but your input_length is only 245. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=122)



Summarized Review: this is a dumb action movie that is just supposed to appeal to a person's shallow senses . a mindless action movie should be at least 1/3 action . most of this movie is filled with unlikeable characters in their 20's, just conversing with each other .
Detected Sentiment: NEGATIVE (Confidence: 1.00)
Detected Emotion: sadness (Confidence: 0.58)
